# **Pre-Processing**

In [ ]:
!git clone https://github.com/salrowili/BioM-Transformers

Cloning into 'BioM-Transformers'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 51 (delta 6), reused 44 (delta 4), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [ ]:
!git clone https://github.com/dmis-lab/biobert
!git clone https://github.com/BioASQ/Evaluation-Measures
!sed -i 's/ix/iloc/g' biobert/biocodes/transform_nbset2bioasqform.py

Cloning into 'biobert'...
remote: Enumerating objects: 329, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 329 (delta 28), reused 8 (delta 3), pack-reused 278
Receiving objects: 100% (329/329), 507.77 KiB | 5.71 MiB/s, done.
Resolving deltas: 100% (190/190), done.
Cloning into 'Evaluation-Measures'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 301 (delta 3), reused 13 (delta 3), pack-reused 268
Receiving objects: 100% (301/301), 1.92 MiB | 11.61 MiB/s, done.
Resolving deltas: 100% (85/85), done.


Establish the connection with TPU. All credits to ALBERT team for creating this code which can be found here https://github.com/google-research/albert/blob/master/albert_glue_fine_tuning_tutorial.ipynb

In [ ]:
%tensorflow_version 1.x
import os
import pprint
import json
import tensorflow as tf

assert "COLAB_TPU_ADDR" in os.environ, "ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!"
TPU_ADDRESS = "grpc://" + os.environ["COLAB_TPU_ADDR"] 
os.environ['TPU_ADDRESS'] ="grpc://" + os.environ["COLAB_TPU_ADDR"] 
TPU_TOPOLOGY = "2x2"
print("TPU address is", TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)


TensorFlow 1.x selected.
TPU address is grpc://10.112.176.186:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 8898500112598780179),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10219461378284026742),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 18102823179197987179),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 11057475274047515050),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17470700658229259392),
 _DeviceAttributes(/job

# **SQuAD2.0** (Skip this step if you plan to use our SQuAD checkpoint)

In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2021-06-21 04:27:39--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  78.7MB/s    in 0.5s    

2021-06-21 04:27:40 (78.7 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2021-06-21 04:27:40--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       

In [ ]:
!gsutil cp  train-v2.0.json gs://biom-transformers/ELECTRA/finetuning_data/squadv2/train.json
!gsutil cp  dev-v2.0.json gs://biom-transformers/ELECTRA/finetuning_data/squadv2/dev.json

Copying file://train-v2.0.json [Content-Type=application/json]...
-
Operation completed over 1 objects/40.2 MiB.                                     
Copying file://dev-v2.0.json [Content-Type=application/json]...
/ [1 files][  4.2 MiB/  4.2 MiB]                                                
Operation completed over 1 objects/4.2 MiB.                                      


In [ ]:
!python BioM-Transformers/electra/run_finetuning.py --data-dir gs://biom-transformers/ELECTRA/ --model-name ELECTRA-SQuAD2.0-Large --hparams '{"model_size": "large", "task_names": ["squad"],"do_train":true,"do_eval":true,"evaluate_bioasq":false,"vocab_file":"gs://biom-transformers/ELECTRA/models/BioM-ELECTRA-Large-TF/vocab.txt","vocab_size":28895,"init_checkpoint":"gs://biom-transformers/ELECTRA/models/BioM-ELECTRA-Large-TF/model.ckpt-434200","num_tpu_cores":8,"use_tpu":true,"tpu_name":"'$TPU_ADDRESS'","save_checkpoints_steps":2000}'

Config: model=ELECTRA-SQuAD2.0-Large, trial 1/1
answerable_classifier True
answerable_uses_start_logits True
answerable_weight 0.5
beam_size 20
data_dir gs://biom-transformers/ELECTRA/
debug False
do_eval True
do_lower_case True
do_test False
do_train True
doc_stride 128
double_unordered True
embedding_size None
eval_batch_size 32
evaluate_bioasq False
gcp_project None
init_checkpoint gs://biom-transformers/ELECTRA/models/BioM-ELECTRA-Large-TF/model.ckpt-434200
iterations_per_loop 1000
joint_prediction True
keep_all_models True
layerwise_lr_decay 0.9
learning_rate 5e-05
log_examples False
max_answer_length 30
max_query_length 64
max_seq_length 512
model_dir gs://biom-transformers/ELECTRA/models/ELECTRA-SQuAD2.0-Large/finetuning_models/squad_model
model_hparam_overrides {}
model_name ELECTRA-SQuAD2.0-Large
model_size large
n_best_size 20
n_writes_test 5
num_tpu_cores 8
num_train_epochs 2.0
num_trials 1
predict_batch_size 32
preprocessed_data_dir gs://biom-transformers/ELECTRA/models/ELE

# **Pre-Processing BioASQ dataset**

Upload the pre-processed dataset of of BioASQ from this page https://github.com/dmis-lab/biobert to this notebook.
Look for "Question Answering: (5.23 MB).

In [ ]:
!unzip /content/BioASQ-67b-15Oct2019.zip

Archive:  /content/BioASQ-67b-15Oct2019.zip
   creating: BioASQ-6b/
   creating: BioASQ-6b/test/
   creating: BioASQ-6b/test/Appended-Snippet/
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-1-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-2-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-3-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-4-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-factoid-6b-snippet-5-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-list-6b-snippet-1-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-list-6b-snippet-2-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-list-6b-snippet-3-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioASQ-test-list-6b-snippet-4-2sent.json  
  inflating: BioASQ-6b/test/Appended-Snippet/BioAS

In [ ]:
!unzip /content/Task7BGoldenEnriched.zip

Archive:  /content/Task7BGoldenEnriched.zip
   creating: Task7BGoldenEnriched/
  inflating: Task7BGoldenEnriched/7B1_golden.json  
  inflating: Task7BGoldenEnriched/7B2_golden.json  
  inflating: Task7BGoldenEnriched/7B3_golden.json  
  inflating: Task7BGoldenEnriched/7B4_golden.json  
  inflating: Task7BGoldenEnriched/7B5_golden.json  


Here we are copying the dataset to our bucket. You need to maintain this path finetuning_data/bioasq-squadv2/test[1-5].json

In [ ]:
!gsutil cp BioASQ-7b/train/Full-Abstract/BioASQ-train-factoid-7b-full-annotated.json gs://biom-transformers/ELECTRA/finetuning_data/bioasq-squadv2/train.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-1.json gs://biom-transformers/ELECTRA/finetuning_data/bioasq-squadv2/test1.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-2.json gs://biom-transformers/ELECTRA/finetuning_data/bioasq-squadv2/test2.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-3.json gs://biom-transformers/ELECTRA/finetuning_data/bioasq-squadv2/test3.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-4.json gs://biom-transformers/ELECTRA/finetuning_data/bioasq-squadv2/test4.json
!gsutil cp /content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-5.json gs://biom-transformers/ELECTRA/finetuning_data/bioasq-squadv2/test5.json

Copying file://BioASQ-7b/train/Full-Abstract/BioASQ-train-factoid-7b-full-annotated.json [Content-Type=application/json]...
/ [1 files][ 10.4 MiB/ 10.4 MiB]                                                
Operation completed over 1 objects/10.4 MiB.                                     
Copying file:///content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-1.json [Content-Type=application/json]...
/ [1 files][159.8 KiB/159.8 KiB]                                                
Operation completed over 1 objects/159.8 KiB.                                    
Copying file:///content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-2.json [Content-Type=application/json]...
/ [1 files][104.8 KiB/104.8 KiB]                                                
Operation completed over 1 objects/104.8 KiB.                                    
Copying file:///content/BioASQ-7b/test/Full-Abstract/BioASQ-test-factoid-7b-3.json [Content-Type=application/json]...
/ [1 files][151.1 KiB/151.1 KiB] 

In [ ]:
!mkdir  temp 
!mkdir  temp/preds/ 
!mkdir  temp/out/

# **Training and Testing BioM-ELECTRA-Large-SQuAD on BioASQ7B**

Here we are using the checkpoint that we fine-tuned early on SQuAD2.0 task . Choose the path to this checkpoint+vocab file using init_checkpoint and vocab_file variables .

In [ ]:
!python BioM-Transformers/electra/run_finetuning.py --data-dir gs://biom-transformers/ELECTRA/ --model-name ELECTRA-BioASQ7B-Large --hparams '{"model_size": "large", "task_names": ["bioasq-squadv2"],"do_train":true,"do_eval":false,"evaluate_bioasq":true,"learning_rate":2e-5 ,"vocab_file":"gs://biom-transformers/ELECTRA/models/BioM-ELECTRA-TF-SQuAD2.0/vocab.txt","vocab_size":28895,"train_batch_size":24,"num_train_epochs":4,"init_checkpoint":"gs://biom-transformers/ELECTRA/models/BioM-ELECTRA-TF-SQuAD2.0/model.ckpt-8186","num_tpu_cores":8,"use_tpu":true,"tpu_name":"'$TPU_ADDRESS'","save_checkpoints_steps":10000}'

Config: model=ELECTRA-BioASQ7B-Large, trial 1/1
answerable_classifier True
answerable_uses_start_logits True
answerable_weight 0.5
beam_size 20
data_dir gs://biom-transformers/ELECTRA/
debug False
do_eval False
do_lower_case True
do_test False
do_train True
doc_stride 128
double_unordered True
embedding_size None
eval_batch_size 32
evaluate_bioasq True
gcp_project None
init_checkpoint gs://biom-transformers/ELECTRA/models/BioM-ELECTRA-TF-SQuAD2.0/model.ckpt-8186
iterations_per_loop 1000
joint_prediction True
keep_all_models True
layerwise_lr_decay 0.9
learning_rate 2e-05
log_examples False
max_answer_length 30
max_query_length 64
max_seq_length 512
model_dir gs://biom-transformers/ELECTRA/models/ELECTRA-BioASQ7B-Large/finetuning_models/bioasq-squadv2_model
model_hparam_overrides {}
model_name ELECTRA-BioASQ7B-Large
model_size large
n_best_size 20
n_writes_test 5
num_tpu_cores 8
num_train_epochs 4
num_trials 1
predict_batch_size 32
preprocessed_data_dir gs://biom-transformers/ELECTRA/mo

In [ ]:
%%shell
rm  temp/* >/dev/null 2>&1
rm  temp/*/* >/dev/null 2>&1
echo "Result of BioASQ on Five Different Batches"
echo "Strict   Lenient   MRR"
for i in 1 2 3 4 5;
do rm temp/out/* >/dev/null 2>&1;
gsutil -q cp gs://biom-transformers/ELECTRA/models/ELECTRA-BioASQ7B-Large/results/bioasq-squadv2_qa/bioasq-squadv2-test"$i"_nbest.json temp/preds;
python3 biobert/biocodes/transform_nbset2bioasqform.py --nbest_path=temp/preds/bioasq-squadv2-test"$i"_nbest.json --output_path=temp/out;java -Xmx10G -cp $CLASSPATH:Evaluation-Measures/flat/BioASQEvaluation/dist/BioASQEvaluation.jar evaluation.EvaluatorTask1b -phaseB -e 5 Task7BGoldenEnriched/7B"$i"_golden.json temp/out/BioASQform_BioASQ-answer.json | awk '{printf "%.2f    %.2f     %.2f\n", $2*100, $3*100, $4*100}';
done;

Result of BioASQ on Five Different Batches
Strict   Lenient   MRR
51.28    56.41     53.08
56.00    64.00     59.00
41.38    51.72     45.98
64.71    82.35     72.55
37.14    51.43     42.62


If you want to see BioASQ predictions for each patch go to temp/preds